## SemEval 2019 Task 4 - Extra Preprocessing Steps Exploration

Jonathan Miller and Negar Adyaniyazdi, VCU, CMSC516, Fall 2018

Goal: Basic exploratory text analysis on a random sample of 10,000 observations

In [5]:
import pandas as pd

DATA_PATH = '../data/'
DATA_PROCESSED_PATH = DATA_PATH + 'processed/'

df = pd.read_csv(DATA_PROCESSED_PATH + 'train10000_0.csv')

In [6]:
df.head()

,id,published-at,title,hyperpartisan,bias,url,labeled-by,article_text,preprocessed_text,tokens
0,310969,2008-12-10,Sweden's ailing auto industry,False,left-center,https://pri.org/stories/2008-12-10/swedens-ail...,publisher,Sweden's ailing auto industry SL says a bailou...,sweden ail auto industry sl say bailout swedis...,"['sweden', 'ail', 'auto', 'industry', 'sl', 's..."
1,945112,NaN,Kidnapped Canadian Mining Executive is Now a P...,True,left,http://occupy.com/article/kidnapped-canadian-m...,publisher,Kidnapped Canadian Mining Executive is Now a P...,kidnap canadian mining executive pawn colombia...,"['kidnap', 'canadian', 'mining', 'executive', ..."
2,519393,2017-12-31,Separate car wrecks claim father and son withi...,False,least,https://apnews.com/64fbe3af704a402aa1f1ceb071a...,publisher,Separate car wrecks claim father and son withi...,separate car wreck claim father son within wee...,"['separate', 'car', 'wreck', 'claim', 'father'..."
3,42462,2017-03-09,A GOP Congressman Just Spent 6 Minutes Defendi...,True,left,https://motherjones.com/politics/2017/03/dana-...,publisher,A GOP Congressman Just Spent 6 Minutes Defendi...,gop congressman spend minute defend vladimir p...,"['gop', 'congressman', 'spend', 'minute', 'def..."
4,736303,2017-11-23,No more ?fighting ISIS?? US to stay in Syria t...,False,right-center,https://newsline.com/no-more-fighting-isis-us-...,publisher,No more ?fighting ISIS?? US to stay in Syria t...,no fight isis us stay syria prevent win assad ...,"['no', 'fight', 'isis', 'us', 'stay', 'syria',..."


In [7]:
df['hyperpartisan'].value_counts(normalize=True)

True     0.5054
False    0.4946
Name: hyperpartisan, dtype: float64

In [8]:
df['bias'].value_counts(normalize=True)

least           0.2781
right           0.2531
left            0.2523
left-center     0.1410
right-center    0.0755
Name: bias, dtype: float64

In [16]:
import gensim
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
DATA_EXTERNAL_PATH = DATA_PATH + 'external/'
word2vec_path = DATA_EXTERNAL_PATH + "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [12]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [17]:
embeddings = get_word2vec_embeddings(word2vec, df)
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(embeddings, list_labels, 
                                                                                        test_size=0.2, random_state=40)

NameError: name 'list_labels' is not defined